# Base Libraries Importation 

In [3]:
!pip install matplotlib
!pip install seaborn
!pip install datetime

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
import csv
import json
import re
import logging
import unicodedata
import datetime as dt
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


# Folder Definition

In [5]:
# Folder Creation
Processed_path = f"./zhaopin/datasets/processed/" # Processed Data File(s) Exist Here 
records_path = f"./zhaopin/datasets/records/" # Raw Data File Exists Here
garbage_path =f"./zhaopin/datasets/garbage/" # Garbage File Exists Here
final_path = f"./zhaopin/datasets/to-be-ingested/" # Final Clean Data File Exists Here

# Create the folder
# os.makedirs(Processed_path, exist_ok=True)
# os.makedirs(records_path, exist_ok=True)
# os.makedirs(garbage_path, exist_ok=True)
# os.makedirs(final_path, exist_ok=True)

# Logging Configuration 

In [6]:

# Logging configuration
log_file_path = f'./zhaopin/datasets/logs/zhaopin_processing.log'
os.makedirs(os.path.dirname(log_file_path), exist_ok=True)  # Ensure the log directory exists

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler(log_file_path, mode='w', encoding='utf-8')]
)

logging.info("Logging has been set up. All logs will be saved to the file.")



# Analysis & Stats (For Post Validation Testing)

In [7]:
def analyze_dataset_chunks(folder_path, report_file, threshold=3, start_file=0, num_files=None):
    """
    Analyze dataset chunks for statistics such as total rows, rows with missing values, duplicates, and special characters.
    Optimized for faster processing.
    """

    # Ensure the report folder exists
    report_folder = os.path.dirname(report_file)
    os.makedirs(report_folder, exist_ok=True)

    # Prepare to store chunk-level statistics
    chunk_stats = []

    # Prepare for overall totals
    overall_totals = {
        "Total Rows": 0,
        "Total Rows with Missing Columns": 0,
        "Total Duplicate Rows": 0,
        "Special Characters Found": Counter()
    }

    # Get list of files and apply slicing based on start_file and num_files
    all_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")])
    selected_files = all_files[start_file:start_file + num_files] if num_files else all_files[start_file:]

    print(f"Dataset Analysis: total files to process: {len(selected_files)}")
    logging.info(f"Dataset Analysis: total files to process: {len(selected_files)}")

    # Define regex for special characters once (with capture group)
    special_char_pattern = r"([^a-zA-Z0-9\s])"  # Add capture group

    # Process each selected CSV file in the folder
    for idx, filename in enumerate(selected_files, start=1):
        file_path = os.path.join(folder_path, filename)
        try:
            print(f"[{idx}/{len(selected_files)}] Processing file: {filename}")
            logging.info(f"[{idx}/{len(selected_files)}] Processing file: {filename}")
            
            # Read the CSV file
            df = pd.read_csv(file_path)

            # Analyze chunk statistics
            total_rows = len(df)
            missing_rows = df.isnull().sum(axis=1).ge(threshold).sum()
            duplicate_rows = df.duplicated().sum()

            # Identify special characters (vectorized operation)
            special_chars_found = Counter()
            for col in df.columns:
                col_special_chars = df[col].astype(str).str.extractall(special_char_pattern).stack().value_counts()
                special_chars_found.update(col_special_chars.to_dict())

            # Append chunk statistics
            chunk_stats.append({
                "File": filename,
                "Total Rows": total_rows,
                "Rows with Missing Columns (> threshold)": missing_rows,
                "Duplicate Rows": duplicate_rows,
                "Special Characters Found": ', '.join(special_chars_found.keys()),
                "Special Character Count": sum(special_chars_found.values())
            })

            # Update overall totals
            overall_totals["Total Rows"] += total_rows
            overall_totals["Total Rows with Missing Columns"] += missing_rows
            overall_totals["Total Duplicate Rows"] += duplicate_rows
            overall_totals["Special Characters Found"].update(special_chars_found)

        except pd.errors.EmptyDataError:
            print(f"Empty file skipped: {filename}")
            logging.error(f"Empty file skipped: {filename}")
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            logging.error(f"Error processing file {filename}: {e}")

    # Save chunk-level statistics to CSV
    chunk_stats_df = pd.DataFrame(chunk_stats)
    chunk_stats_report = os.path.join(report_folder, "chunk_statistics.csv")
    chunk_stats_df.to_csv(chunk_stats_report, index=False)
    print(f"Chunk-level statistics saved to: {chunk_stats_report}")
    logging.info(f"Chunk-level statistics saved to: {chunk_stats_report}")

    # Save overall totals to CSV
    overall_totals_report = os.path.join(report_folder, "overall_totals.csv")
    overall_totals_df = pd.DataFrame([{
        "Total Rows": overall_totals["Total Rows"],
        "Rows with Missing Columns (> threshold)": overall_totals["Total Rows with Missing Columns"],
        "Duplicate Rows": overall_totals["Total Duplicate Rows"],
        "Special Characters Found": ', '.join(overall_totals["Special Characters Found"].keys()),
        "Special Character Count": sum(overall_totals["Special Characters Found"].values())
    }])
    overall_totals_df.to_csv(overall_totals_report, index=False)
    print(f"Combined chunk statistics saved to: {overall_totals_report}")
    logging.info(f"Combined chunk statistics saved to: {overall_totals_report}")

folder_path = "./zhaopin/datasets/records/"
report_file = "./zhaopin/datasets/logs/reports/zhaopin_dataset_statistics.csv"

# Set the start file index and number of files to process
start_file = 0  # Start from the 1st file (index 0)
num_files = None   # Process 1 file

# Analyze the dataset chunks
analyze_dataset_chunks(folder_path, report_file, threshold=3, start_file=start_file, num_files=num_files)

Dataset Analysis: total files to process: 1
[1/1] Processing file: zhaopin.csv


C:\Users\max_m\AppData\Local\Temp\ipykernel_25676\1431167071.py:40: DtypeWarning: Columns (0,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Chunk-level statistics saved to: ./zhaopin/datasets/logs/reports\chunk_statistics.csv
Combined chunk statistics saved to: ./zhaopin/datasets/logs/reports\overall_totals.csv


# Column Stats

In [8]:
# Function to create statistics for each column. 
# Two CSvs produced, one which shows stats per chunk, the other with stats for all chunks in folder.
def analyze_csv_columns(folder_path, report_file, start_file=0, num_files=None):
    """
    Analyze CSV files in a folder for column-level statistics and combined totals across all chunks.
    
    Args:
        folder_path (str): Path to the folder containing CSV files.
        report_file (str): Path to save the column statistics report.
        start_file (int): Index of the file to start processing from.
        num_files (int or None): Number of files to process. If None, processes all files starting from start_file.
        
    Returns:
        None
    """
    
    # Ensure the report folder exists
    report_folder = os.path.dirname(report_file)
    os.makedirs(report_folder, exist_ok=True)
    
    # Prepare to store column statistics and combined totals
    column_stats = []
    combined_totals = defaultdict(lambda: {
        "Total Values": 0,
        "Missing Values": 0,
        "Duplicate Values": 0,
        "Special Characters Found": Counter(),
        "Special Character Count": 0
    })

    # Helper function to find special characters
    def find_special_characters(value):
        special_chars = re.findall(r"[^a-zA-Z0-9\s]", str(value))
        return special_chars if special_chars else []

    # Get list of files and apply slicing based on start_file and num_files
    all_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")])
    selected_files = all_files[start_file:start_file + num_files] if num_files else all_files[start_file:]

    print(f"Column by Column Analysis: total files to process: {len(selected_files)}")
    logging.info(f"Column by Column Analysis: total files to process: {len(selected_files)}")
    
    # Process each selected CSV file in the folder
    for idx, filename in enumerate(selected_files, start=1):
        file_path = os.path.join(folder_path, filename)
        try:
            print(f"[{idx}/{len(selected_files)}] Processing file: {filename}")
            logging.info(f"[{idx}/{len(selected_files)}] Processing file: {filename}")
            
            # Read the CSV file
            df = pd.read_csv(file_path)
            
            # Analyze each column
            for col in df.columns:
                total_values = len(df[col])
                missing_values = df[col].isnull().sum()
                duplicate_values = df[col].duplicated().sum()
                special_chars_found = Counter()
                
                # Find special characters in the column
                for value in df[col]:
                    special_chars = find_special_characters(value)
                    special_chars_found.update(special_chars)
                
                # Append statistics for the column in this file
                column_stats.append({
                    "File": filename,
                    "Column": col,
                    "Total Values": total_values,
                    "Missing Values": missing_values,
                    "Duplicate Values": duplicate_values,
                    "Special Characters Found": ', '.join(set(special_chars_found)),
                    "Special Character Count": sum(special_chars_found.values())
                })

                # Update combined totals
                combined_totals[col]["Total Values"] += total_values
                combined_totals[col]["Missing Values"] += missing_values
                combined_totals[col]["Duplicate Values"] += duplicate_values
                combined_totals[col]["Special Characters Found"].update(special_chars_found)
                combined_totals[col]["Special Character Count"] += sum(special_chars_found.values())
        
        except pd.errors.EmptyDataError:
            print(f"Empty file skipped: {filename}")
            logging.warning(f"Empty file skipped: {filename}")
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            logging.error(f"Error processing file {filename}: {e}")

    # Save column statistics for individual files to CSV
    column_stats_df = pd.DataFrame(column_stats)
    column_stats_report = report_file
    column_stats_df.to_csv(column_stats_report, index=False)
    print(f"Column statistics saved to: {column_stats_report}")
    logging.info(f"Column statistics saved to: {column_stats_report}")

    # Save combined totals to a separate CSV
    combined_stats_data = []
    for col, stats in combined_totals.items():
        combined_stats_data.append({
            "Column": col,
            "Total Values": stats["Total Values"],
            "Missing Values": stats["Missing Values"],
            "Duplicate Values": stats["Duplicate Values"],
            "Special Characters Found": ', '.join(set(stats["Special Characters Found"])),
            "Special Character Count": stats["Special Character Count"]
        })

    combined_stats_df = pd.DataFrame(combined_stats_data)
    combined_stats_report = os.path.splitext(report_file)[0] + "_combined_totals.csv"
    combined_stats_df.to_csv(combined_stats_report, index=False)
    print(f"Combined column statistics saved to: {combined_stats_report}")
    logging.info(f"Combined column statistics saved to: {combined_stats_report}")

# Example Usage
folder_path = "./zhaopin/datasets/to-be-ingested/"
report_file = "./zhaopin/datasets/reports/zhaopin_column_statistics.csv"

# Set the start file index and number of files to process
start_file = 0  # Start from the first file
num_files = None  # Process all files

analyze_csv_columns(folder_path, report_file, start_file=start_file, num_files=num_files)

Column by Column Analysis: total files to process: 1
[1/1] Processing file: zhaopin-cleaned_output.csv
Column statistics saved to: ./zhaopin/datasets/reports/zhaopin_column_statistics.csv
Combined column statistics saved to: ./zhaopin/datasets/reports/zhaopin_column_statistics_combined_totals.csv


# Custom Column Analysis

In [9]:
#Analyze CSV files in a folder based on specified columns and functions.
#Adds a special characters summary to the combined statistics file without increasing processing time.

# Validation functions
def is_alphabetical(value):
    return str(value).isalpha()

def is_alphanumeric(value):
    return str(value).isalnum()

def is_numeric(value):
    return str(value).isdigit()

def contains_special_chars(value):
    return bool(re.search(r"[^a-zA-Z0-9\s]", str(value)))

def is_valid_date(value):
    if pd.isnull(value) or value == "":
        return False
    if not re.match(r"^\d{1,4}[-/]\d{1,2}[-/]\d{1,4}$", str(value)):
        return False
    try:
        pd.to_datetime(value, errors="raise")
        return True
    except (ValueError, TypeError):
        return False

def is_valid_email(value):
    return bool(re.match(r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$", str(value)))

def is_missing(value):
    return pd.isnull(value)

def is_duplicate(column):
    seen = set()
    duplicates = []
    for value in column:
        if value in seen:
            duplicates.append(True)
        else:
            duplicates.append(False)
            seen.add(value)
    return pd.Series(duplicates, index=column.index)

def list_special_characters(value):
    special_chars = re.findall(r"[^a-zA-Z0-9\s]", str(value))
    return ", ".join(special_chars) if special_chars else None

def extract_unique_values(list_of_strings):
  """Extracts unique values from a list of strings, splitting each string by commas.

  Args:
    list_of_strings: A list of strings.

  Returns:
    A set of unique values.
  """

  all_values = []
  for string in list_of_strings:
    if string is not None:
      values = string.split(', ')
      all_values.extend(values)

  unique_values = set(all_values)
  return unique_values

def analyze_csv_columns_with_functions(folder_path, report_file, columns, functions, start_file=0, num_files=None):
    """
    Analyze CSV files in a folder based on specified columns and functions.
    Adds a special characters summary to the combined statistics file without increasing processing time.
    """
    # Ensure the report folder exists
    report_folder = os.path.dirname(report_file)
    os.makedirs(report_folder, exist_ok=True)

    # Prepare to store statistics
    column_stats = []
    combined_totals = defaultdict(lambda: defaultdict(int))
    special_characters_per_file = []
    aggregated_special_characters = defaultdict(set)  # To store cumulative special characters for each column

    # Get list of files and apply slicing
    all_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")])
    selected_files = all_files[start_file:start_file + num_files] if num_files else all_files[start_file:]

    print(f"Custom Analysis: Processing {len(selected_files)} files. Analyzing columns: {columns}.")
    logging.info(f"Custom Analysis: Processing {len(selected_files)} files. Analyzing columns: {columns}.")

    # Process each selected file
    for idx, filename in enumerate(selected_files, start=1):
        file_path = os.path.join(folder_path, filename)
        try:
            print(f"[{idx}/{len(selected_files)}] Processing file: {filename}")
            logging.info(f"[{idx}/{len(selected_files)}] Processing file: {filename}")

            # Read the CSV file
            df = pd.read_csv(file_path)

            # Analyze specified columns
            for col in columns:
                if col not in df.columns:
                    print(f"Column {col} not found in {filename}. Skipping.")
                    logging.warning(f"Column {col} not found in {filename}. Skipping.")
                    continue

                for func_name, func in functions.items():
                    try:
                        if func_name == "Is Duplicate":
                            results = is_duplicate(df[col])
                        elif func_name == "List Special Characters":
                            # List special characters for this file and column
                            results = df[col].apply(func)
                            special_characters_summary = extract_unique_values(results)
                            aggregated_special_characters[col].update(special_characters_summary)  # Aggregate
                            special_characters_per_file.append({
                                "File": filename,
                                "Column": col,
                                "Special Characters Summary": str(special_characters_summary)
                            })
                            column_stats.append({
                                "File": filename,
                                "Column": col,
                                "Function": func_name,
                                "Special Characters Summary": str(special_characters_summary)
                            })
                            continue
                        else:
                            results = df[col].apply(func)

                        total_values = len(df[col])
                        matches = results.sum() if func_name != "Is Duplicate" else results.sum()
                        non_matches = total_values - matches

                        # Append statistics for the column in this file
                        column_stats.append({
                            "File": filename,
                            "Column": col,
                            "Function": func_name,
                            "Matches": matches,
                            "Non-Matches": non_matches,
                            "Total Values": total_values
                        })

                        # Update combined totals
                        combined_totals[col][f"{func_name}_Matches"] += matches
                        combined_totals[col][f"{func_name}_Non-Matches"] += non_matches
                        combined_totals[col]["Total Values"] += total_values
                        
                    except Exception as e:
                        print(f"Error applying function {func_name} on column {col} in {filename}: {e}")
                        logging.error(f"Error applying function {func_name} on column {col} in {filename}: {e}")

        except pd.errors.EmptyDataError:
            print(f"Empty file skipped: {filename}")
            logging.warning(f"Empty file skipped: {filename}")
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            logging.error(f"Error processing file {filename}: {e}")

    # Save column statistics for individual files to CSV
    column_stats_df = pd.DataFrame(column_stats)
    column_stats_df.to_csv(report_file, index=False)
    print(f"Column statistics saved to: {report_file}")
    logging.info(f"Column statistics saved to: {report_file}")

    # Generate special characters summary for the combined statistics file
    unique_special_characters = set()
    for entry in special_characters_per_file:
        special_chars = eval(entry["Special Characters Summary"])  # Safely parse string dictionary
        unique_special_characters.update(special_chars)

    # Save combined totals and special characters summary to a separate CSV
    combined_stats_data = []
    for col, stats in combined_totals.items():
        combined_stats_data.append({
            "Column": col,
            **stats,
            "Special Characters (Aggregated)": ", ".join(sorted(aggregated_special_characters[col]))
        })

    # Append unique special characters summary for all columns
    combined_stats_data.append({
        "Column": "All Columns",
        "Unique Special Characters": ", ".join(sorted(unique_special_characters)),
        #"Total Special Characters": len(unique_special_characters)
    })

    combined_stats_df = pd.DataFrame(combined_stats_data)
    combined_stats_report = os.path.splitext(report_file)[0] + "_combined.csv"
    combined_stats_df.to_csv(combined_stats_report, index=False)
    print(f"Combined column statistics saved to: {combined_stats_report}")
    logging.info(f"Combined column statistics saved to: {combined_stats_report}")

# Example Usage
folder_path = "./zhaopin/datasets/to-be-ingested/"
report_file = "./zhaopin/datasets/logs/reports/zhaopin_custom_column_stats.csv"

columns_to_analyze = ["name","email"]
analysis_functions = {
    "Is Valid Email": is_valid_email,
    #"Contains Special Characters": contains_special_chars,
    "List Special Characters": list_special_characters,
    "Is Missing": is_missing,
    "Is Duplicate": None,
}

start_file = 0  # Start from the first file
num_files = None  # Process all files

analyze_csv_columns_with_functions(folder_path, report_file, columns_to_analyze, analysis_functions, start_file, num_files)

Custom Analysis: Processing 1 files. Analyzing columns: ['name', 'email'].
[1/1] Processing file: zhaopin-cleaned_output.csv
Column statistics saved to: ./zhaopin/datasets/logs/reports/zhaopin_custom_column_stats.csv
Combined column statistics saved to: ./zhaopin/datasets/logs/reports/zhaopin_custom_column_stats_combined.csv
